In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

### KuCoin API Base URL + Parameters:

#### Format: OCHL; oldest to newest

https://api.kucoin.com/api/v1/market/candles "?"

* symbol: Trading pair (e.g., BTC-USDT).
* type: Time interval (e.g., 1min, 5min, 15min, 1hour, 12hour, 1day).
* startAt: Start time in UNIX timestamp (seconds since epoch).
* endAt: End time in UNIX timestamp (seconds since epoch).
* limit: Number of candles to fetch (max 1500).

https://api.kucoin.com/api/v1/market/candles?symbol=BTC-USDT&type=12hour&startAt=1717449600&endAt=1717536000&limit=1000

ps. "Klines" refers to candlestick (OHLC) data
ps. the ? separates the base URL from the query parameters
ps. Use the timestamp of the day at 00:00 as the startTime and timestamp of the next day at 00:00 as the endTime to ensure the last day is complete.
ps. use https://www.epochconverter.com/ to convert time to UNIX

### Fetching Data

In [5]:
def fetch_kucoin_data(symbol, interval, startAt, endAt):
    base_url = "https://api.kucoin.com/api/v1/market/candles" # the base url and parameters 
    params = {
        "symbol": symbol,
        "type": interval,
        "startAt": startAt,
        "endAt": endAt
    }
    response = requests.get(base_url, params=params) # requesting the data from the API via requests library
    data = response.json()
    candles = data["data"]
    return pd.DataFrame({ # create a dataframe only leaving the closing prices 
        "time": [datetime.fromtimestamp(int(entry[0])) for entry in candles], # change UNIX time to human time.
        "close": [float(entry[2]) for entry in candles] # fetch the close price which has index 2
    }).iloc[::-1].reset_index(drop=True) #reverse the order bc in Kucoin the datapoints are oldest to newest

### Analyzing Data

In [7]:
def analyze_data(master_data, test_data):
    up_success = 0
    down_success = 0
    up_total = 0
    down_total = 0

    for i in range(0, len(master_data), 2): #stop when there is no more datapoints
        if i + 1 >= len(master_data):
            break

        t0_master_price = master_data.iloc[i]['close'] # calculate percent change 
        t1_master_price = master_data.iloc[i + 1]['close']
        master_change = ((t1_master_price - t0_master_price) / t0_master_price) * 100

        if abs(master_change) > 1: #test primary tokens t_0 against secondary's t_1
            corresponding_day = i // 2
            if corresponding_day + 1 >= len(test_data):
                continue

            t0_test_price = test_data.iloc[corresponding_day]['close'] # create test variables 
            t1_test_price = test_data.iloc[corresponding_day + 1]['close']

            if master_change > 1: # main logic
                up_total += 1
                if t1_test_price > t0_test_price:
                    up_success += 1
            elif master_change < -1:
                down_total += 1
                if t1_test_price < t0_test_price:
                    down_success += 1

    up_success_rate = (up_success / up_total * 100) if up_total > 0 else 0 # count success rates
    down_success_rate = (down_success / down_total * 100) if down_total > 0 else 0

    print("Primary moved +1%: " + str(up_total) + " times") # print the results 
    print("Primary moved -1%: " + str(down_total) + " times")
    print("Success for up movements: " + str(up_success) + " times")
    print("Success for down movements: " + str(down_success) + " times")
    
    return up_success_rate, down_success_rate

### User Inputs...

In [9]:
master_data = fetch_kucoin_data("BTC-USDT", "12hour", 1671824237, 1734982637) #watch out for the syntax of inputs
test_data = fetch_kucoin_data("WIF-USDT", "12hour", 1671824237, 1734982637)

In [10]:
up_success_rate, down_success_rate = analyze_data(master_data, test_data)

Primary moved +1%: 155 times
Primary moved -1%: 148 times
Success for up movements: 69 times
Success for down movements: 75 times


In [11]:
print(f"Success rate for primary up and secondary up: {up_success_rate:.2f}%")
print(f"Success rate for primary down and secondary down: {down_success_rate:.2f}%")

Success rate for primary up and secondary up: 44.52%
Success rate for primary down and secondary down: 50.68%


### Human date to UNIX for the input

In [13]:
human_start_date = "2023-8-10 20:00:00"
unix_start_date = int(datetime.strptime(human_start_date, "%Y-%m-%d %H:%M:%S").timestamp())
print(f"The UNIX timestamp for {human_start_date} is {unix_start_date} seconds.")

The UNIX timestamp for 2023-8-10 20:00:00 is 1691712000 seconds.


In [14]:
master_data

,time,close
0,2022-12-23 19:00:00,16826.2
1,2022-12-24 07:00:00,16836.1
2,2022-12-24 19:00:00,16822.0
3,2022-12-25 07:00:00,16832.8
4,2022-12-25 19:00:00,16860.0
...,...,...
1457,2024-12-21 07:00:00,97280.2
1458,2024-12-21 19:00:00,97099.4
1459,2024-12-22 07:00:00,95186.2
1460,2024-12-22 19:00:00,95936.2


In [15]:
test_data

,time,close
0,2024-01-18 07:00:00,0.3629
1,2024-01-18 19:00:00,0.3860
2,2024-01-19 07:00:00,0.4021
3,2024-01-19 19:00:00,0.3487
4,2024-01-20 07:00:00,0.3381
...,...,...
676,2024-12-21 07:00:00,1.8885
677,2024-12-21 19:00:00,1.9667
678,2024-12-22 07:00:00,1.9193
679,2024-12-22 19:00:00,1.9024
